<a href="https://colab.research.google.com/github/bhadreshpsavani/UnderstandingNLP/blob/master/TFLiteExperimentsQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import


In [49]:
!pip install -q transformers

In [50]:
import tensorflow as tf
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
print(tf.__version__)

2.3.0


## Get Distilbert Model

In [3]:
from transformers import DistilBertTokenizer, TFDistilBertForQuestionAnswering
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-distilled-squad')
model = TFDistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad', return_dict=True)

All model checkpoint layers were used when initializing TFDistilBertForQuestionAnswering.

All the layers of TFDistilBertForQuestionAnswering were initialized from the model checkpoint at distilbert-base-uncased-distilled-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForQuestionAnswering for predictions without further training.


In [4]:
question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
input_dict = tokenizer(question, text, padding='max_length', max_length=512,  truncation=True,  return_tensors='tf')
outputs = model(input_dict['input_ids'])
start_logits = outputs.start_logits
end_logits = outputs.end_logits
all_tokens = tokenizer.convert_ids_to_tokens(input_dict["input_ids"].numpy()[0])
answer = ' '.join(all_tokens[tf.math.argmax(start_logits, 1)[0] : tf.math.argmax(end_logits, 1)[0] +1])
answer

'nice puppet'

In [5]:
outputs

TFQuestionAnsweringModelOutput([('start_logits',
                                 <tf.Tensor: shape=(1, 512), dtype=float32, numpy=
                                 array([[-0.17469537, -3.5080047 , -6.1385064 , -5.5235963 , -8.018404  ,
                                         -6.8259974 ,  0.57807636,  3.0221834 , -1.4504491 ,  1.6885171 ,
                                          8.354978  ,  8.624665  ,  5.204956  ,  0.5748353 , -0.95076597,
                                         -2.3498573 , -2.712817  , -3.3959804 , -3.6995823 , -3.6311238 ,
                                         -3.8023012 , -3.7198126 , -3.6485667 , -3.7160215 , -3.5786633 ,
                                         -3.5681887 , -3.5543375 , -3.6937852 , -3.827592  , -3.8174355 ,
                                         -3.9082553 , -3.8810377 , -3.8540561 , -3.8498788 , -3.8819816 ,
                                         -3.8665907 , -3.8555217 , -3.858398  , -3.917903  , -4.016426  ,
                    

## Get data

In [7]:
sample_df=pd.read_csv('test.csv', index_col=0)[:1000]
sample_df.head()

,id,question,context,answers,c_id
0,56be4db0acb8001400a502ec,Which NFL team represented the AFC at Super Bo...,Super Bowl 50 was an American football game to...,"[{'answer_start': 177, 'text': 'Denver Broncos...",0
1,56be4db0acb8001400a502ed,Which NFL team represented the NFC at Super Bo...,Super Bowl 50 was an American football game to...,"[{'answer_start': 249, 'text': 'Carolina Panth...",0
2,56be4db0acb8001400a502ee,Where did Super Bowl 50 take place?,Super Bowl 50 was an American football game to...,"[{'answer_start': 403, 'text': 'Santa Clara, C...",0
3,56be4db0acb8001400a502ef,Which NFL team won Super Bowl 50?,Super Bowl 50 was an American football game to...,"[{'answer_start': 177, 'text': 'Denver Broncos...",0
4,56be4db0acb8001400a502f0,What color was used to emphasize the 50th anni...,Super Bowl 50 was an American football game to...,"[{'answer_start': 488, 'text': 'gold'}, {'answ...",0


In [8]:
sample_df['answers'][3]

"[{'answer_start': 177, 'text': 'Denver Broncos'}, {'answer_start': 177, 'text': 'Denver Broncos'}, {'answer_start': 177, 'text': 'Denver Broncos'}]"

In [9]:
sample_df['input_ids']= sample_df.apply(lambda row: tokenizer(row['question'], row['context'], padding='max_length', max_length=512,  truncation=True,  return_tensors='tf')['input_ids'], axis=1)

In [10]:
def predict(input_ids):
    outputs = model(input_ids)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits
    all_tokens = tokenizer.convert_ids_to_tokens(input_ids.numpy()[0])
    answer = ' '.join(all_tokens[tf.math.argmax(start_logits, 1)[0] : tf.math.argmax(end_logits, 1)[0] +1])
    return answer

In [11]:
%%time
sample_df['answer_text']= sample_df['input_ids'].apply(predict)

CPU times: user 23min 51s, sys: 37.3 s, total: 24min 28s
Wall time: 13min 39s


In [12]:
sample_df['target_text'] = sample_df['answers'].apply(lambda answer: answer.split("}")[0].split(":")[-1])

In [15]:
# sample_df.to_csv('sample_test.csv')

## Mesuring The Score

In [13]:
"""Evaluation utilities."""
import argparse
import collections
import json
import os
import re
import string
import sys
import numpy as np

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()


def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))


def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def make_eval_dict(exact_scores, f1_scores):
    total = len(exact_scores)
    return collections.OrderedDict([("exact", 100.0 * sum(exact_scores.values / total)), ("f1", 100.0 * sum(f1_scores.values / total)),("total", total)])

In [14]:
input = "denver broncos"
output ="/denver broncos'"
compute_exact(input, output)

1

In [15]:
compute_f1(input, output)

1.0

In [16]:
sample_df['exact_match'] = sample_df.apply(lambda row:compute_exact(row['target_text'], row['answer_text']), axis=1)
sample_df['f1_score'] = sample_df.apply(lambda row:compute_f1(row['target_text'], row['answer_text']), axis=1)

In [17]:
make_eval_dict(sample_df['exact_match'], sample_df['f1_score'])

OrderedDict([('exact', 46.000000000000036),
             ('f1', 58.62087773284195),
             ('total', 1000)])

## Configure Model Input:

In [18]:
input_spec = tf.TensorSpec([1, 512], tf.int32)
# model._set_inputs(input_spec, training=False) # for tf < 2.2
model._saved_model_inputs_spec = None # for tf > 2.2
model._set_save_spec(input_spec) # for tf > 2.2
input_spec

TensorSpec(shape=(1, 512), dtype=tf.int32, name=None)

In [19]:
model.inputs

In [20]:
model.save_weights('./tensorflow_distilbert/checkpoint')

# TensorFlow Lite:

## With Normal Converstion:

In [21]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# For normal conversion:
converter.target_spec.supported_ops = [tf.lite.OpsSet.SELECT_TF_OPS]

In [22]:
tflite_model = converter.convert()
open("distilbert.tflite", "wb").write(tflite_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /tmp/tmp5gh7seb6/assets


INFO:absl:Using experimental converter: If you encountered a problem please file a bug. You can opt-out by setting experimental_new_converter=False


265660704

In [23]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="distilbert.tflite")
interpreter.allocate_tensors()

In [24]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [25]:
input_details

[{'dtype': numpy.int32,
  'index': 0,
  'name': 'args_0',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([  1, 512], dtype=int32),
  'shape_signature': array([  1, 512], dtype=int32),
  'sparsity_parameters': {}}]

In [26]:
output_details

[{'dtype': numpy.float32,
  'index': 636,
  'name': 'Identity',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([  1, 512], dtype=int32),
  'shape_signature': array([  1, 512], dtype=int32),
  'sparsity_parameters': {}},
 {'dtype': numpy.float32,
  'index': 634,
  'name': 'Identity_1',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([  1, 512], dtype=int32),
  'shape_signature': array([  1, 512], dtype=int32),
  'sparsity_parameters': {}}]

In [27]:
%%time
interpreter.set_tensor(input_details[0]['index'], input_dict['input_ids'])
interpreter.invoke()
end_logits  = interpreter.get_tensor(output_details[0]['index'])
start_logits = interpreter.get_tensor(output_details[1]['index'])
all_tokens = tokenizer.convert_ids_to_tokens(input_dict["input_ids"].numpy()[0])
answer = ' '.join(all_tokens[tf.math.argmax(start_logits, 1)[0] : tf.math.argmax(end_logits, 1)[0] +1])
answer

CPU times: user 1.23 s, sys: 75.5 ms, total: 1.31 s
Wall time: 724 ms


In [28]:
def predict_tflite(input_ids):
    interpreter.set_tensor(input_details[0]['index'], input_ids)
    interpreter.invoke()
    end_logits  = interpreter.get_tensor(output_details[0]['index'])
    start_logits = interpreter.get_tensor(output_details[1]['index'])
    all_tokens = tokenizer.convert_ids_to_tokens(input_ids.numpy()[0])
    answer = ' '.join(all_tokens[tf.math.argmax(start_logits, 1)[0] : tf.math.argmax(end_logits, 1)[0] +1])
    return answer

In [29]:
%%time
sample_df['tflite_answer'] = sample_df['input_ids'].apply(predict_tflite)

CPU times: user 20min 36s, sys: 45.3 s, total: 21min 22s
Wall time: 11min 20s


In [30]:
sample_df['exact_match'] = sample_df.apply(lambda row:compute_exact(row['target_text'], row['tflite_answer']), axis=1)
sample_df['f1_score'] = sample_df.apply(lambda row:compute_f1(row['target_text'], row['tflite_answer']), axis=1)
make_eval_dict(sample_df['exact_match'], sample_df['f1_score'])

OrderedDict([('exact', 46.000000000000036),
             ('f1', 58.62087773284195),
             ('total', 1000)])

## FP16 Quantization:

In [31]:
# Below Two methods makes models size 4 time smaller
# For conversion with FP16 quantization:
# supports CPUs, GPUs
converter_16 = tf.lite.TFLiteConverter.from_keras_model(model)
converter_16.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter_16.target_spec.supported_types = [tf.float16]
converter_16.optimizations = [tf.lite.Optimize.DEFAULT]
converter_16.experimental_new_converter = True

tflite_model_fp16 = converter_16.convert()
open("distilbert_fp16.tflite", "wb").write(tflite_model_fp16)

# Load the TFLite model and allocate tensors.
interpreter_fp16 = tf.lite.Interpreter(model_path="distilbert_fp16.tflite")
interpreter_fp16.allocate_tensors()

# Get input and output tensors.
input_details = interpreter_fp16.get_input_details()
output_details = interpreter_fp16.get_output_details()

INFO:tensorflow:Assets written to: /tmp/tmptps4osf3/assets


INFO:tensorflow:Assets written to: /tmp/tmptps4osf3/assets
INFO:absl:Using experimental converter: If you encountered a problem please file a bug. You can opt-out by setting experimental_new_converter=False


In [47]:
%%time
interpreter_fp16.set_tensor(input_details[0]['index'], input_dict['input_ids'])
interpreter_fp16.invoke()
output_data_fp16 = interpreter_fp16.get_tensor(output_details[0]['index'])

CPU times: user 1.22 s, sys: 5.92 ms, total: 1.23 s
Wall time: 1.23 s


In [33]:
def predict_tflite_16(input_ids):
    interpreter_fp16.set_tensor(input_details[0]['index'], input_ids)
    interpreter_fp16.invoke()
    end_logits  = interpreter_fp16.get_tensor(output_details[0]['index'])
    start_logits = interpreter_fp16.get_tensor(output_details[1]['index'])
    all_tokens = tokenizer.convert_ids_to_tokens(input_ids.numpy()[0])
    answer = ' '.join(all_tokens[tf.math.argmax(start_logits, 1)[0] : tf.math.argmax(end_logits, 1)[0] +1])
    return answer

In [34]:
%%time
sample_df['answer_text'] = sample_df['input_ids'].apply(predict_tflite_16)

CPU times: user 17min 32s, sys: 2.21 s, total: 17min 34s
Wall time: 17min 27s


In [35]:
sample_df['exact_match'] = sample_df.apply(lambda row:compute_exact(row['target_text'], row['answer_text']), axis=1)
sample_df['f1_score'] = sample_df.apply(lambda row:compute_f1(row['target_text'], row['answer_text']), axis=1)
make_eval_dict(sample_df['exact_match'], sample_df['f1_score'])

OrderedDict([('exact', 46.000000000000036),
             ('f1', 58.62087773284195),
             ('total', 1000)])

## Hybrid Quantization:

In [37]:
# For conversion with hybrid quantization:
# This only support CPU
converter_hy = tf.lite.TFLiteConverter.from_keras_model(model)
converter_hy.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter_hy.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
converter_hy.experimental_new_converter = True

tflite_model_hy = converter_hy.convert()
open("distilbert_hy.tflite", "wb").write(tflite_model_hy)

# Load the TFLite model and allocate tensors.
interpreter_hy = tf.lite.Interpreter(model_path="distilbert_hy.tflite")
interpreter_hy.allocate_tensors()

# Get input and output tensors.
input_details = interpreter_hy.get_input_details()
output_details = interpreter_hy.get_output_details()

INFO:tensorflow:Assets written to: /tmp/tmpz1ogdzpt/assets


INFO:tensorflow:Assets written to: /tmp/tmpz1ogdzpt/assets
INFO:absl:Using experimental converter: If you encountered a problem please file a bug. You can opt-out by setting experimental_new_converter=False


In [48]:
%%time
interpreter_hy.set_tensor(input_details[0]['index'], input_dict['input_ids'])
interpreter_hy.invoke()
output_data_hy = interpreter_hy.get_tensor(output_details[0]['index'])
output_data_hy

CPU times: user 1.92 s, sys: 946 µs, total: 1.92 s
Wall time: 1.92 s


In [40]:
def predict_tflite_hy(input_ids):
    interpreter_hy.set_tensor(input_details[0]['index'], input_ids)
    interpreter_hy.invoke()
    end_logits  = interpreter_hy.get_tensor(output_details[0]['index'])
    start_logits = interpreter_hy.get_tensor(output_details[1]['index'])
    all_tokens = tokenizer.convert_ids_to_tokens(input_ids.numpy()[0])
    answer = ' '.join(all_tokens[tf.math.argmax(start_logits, 1)[0] : tf.math.argmax(end_logits, 1)[0] +1])
    return answer

In [41]:
%%time
sample_df['answer_text'] = sample_df['input_ids'].apply(predict_tflite_hy)

CPU times: user 32min 12s, sys: 1.55 s, total: 32min 14s
Wall time: 32min 7s


In [43]:
sample_df['exact_match'] = sample_df.apply(lambda row:compute_exact(row['target_text'], row['answer_text']), axis=1)
sample_df['f1_score'] = sample_df.apply(lambda row:compute_f1(row['target_text'], row['answer_text']), axis=1)
make_eval_dict(sample_df['exact_match'], sample_df['f1_score'])

OrderedDict([('exact', 45.10000000000004),
             ('f1', 57.11463039269687),
             ('total', 1000)])

## Observations:
For DistilBert For Sequence Classification Model here is Our Observation

|Method|Model Weight Size|Inference Time| F1_Score | Exact Match |
|----- | ----| ----- | ---- | ---- |
| Normal Tensorflow Model | 253.85 Mb | 24min 28s | 58.62 | 46 |
| Normal TFLite | 253.33 Mb| 21min 22s | 58.62 | 46 |
| TFLite with FP16 Quantization | 126.88 Mb | 17min 34s| 58.62 | 46 |
| TFLite With Hybrid Quantization | 64.87 Mb | 32min 14s | 57.12 | 45.1 |

* Support Fix Length Inputs
* Only Support TensorFLow and Keras Model
* TFlite might not support Complex Model input While Inference

## Albert:

In [57]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
tokenizer = AutoTokenizer.from_pretrained("ahotrod/albert_xxlargev1_squad2_512")
model = AutoModelForQuestionAnswering.from_pretrained("ahotrod/albert_xxlargev1_squad2_512")

In [60]:
question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
input_dict = tokenizer(question, text, padding='max_length', max_length=512,  truncation=True,  return_tensors='tf')
input_dict

{'input_ids': <tf.Tensor: shape=(1, 512), dtype=int32, numpy=
array([[    2,    72,    23,  2170, 27674,    60,     3,  2170, 27674,
           23,    21,  2210, 10956,     3,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0, 

In [61]:
outputs = model(input_dict['input_ids'])
start_logits = outputs.start_logits
end_logits = outputs.end_logits
all_tokens = tokenizer.convert_ids_to_tokens(input_dict["input_ids"].numpy()[0])
answer = ' '.join(all_tokens[tf.math.argmax(start_logits, 1)[0] : tf.math.argmax(end_logits, 1)[0]+1])
answer

AttributeError: ignored

## Roberta:

In [ ]:
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = TFRobertaForSequenceClassification.from_pretrained('roberta-base', return_dict=True)

inputs = tokenizer("Hello, my dog is cute", return_tensors="tf")
inputs["labels"] = tf.reshape(tf.constant(1), (-1, 1)) # Batch size 1

outputs = model(inputs)
loss = outputs.loss
logits = outputs.logits

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaForSequenceClassification: ['lm_head']
- This IS expected if you are initializing TFRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
input_spec = tf.TensorSpec([1, 8], tf.int32)
model._set_inputs(input_spec, training=False) # for tf < 2.2
# model._saved_model_inputs_spec = None # for tf > 2.2
# model._set_save_spec(input_spec) # for tf > 2.2
input_spec

TensorSpec(shape=(1, 8), dtype=tf.int32, name=None)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# For normal conversion:
converter.target_spec.supported_ops = [tf.lite.OpsSet.SELECT_TF_OPS]

tflite_model = converter.convert()
open("roberta.tflite", "wb").write(tflite_model)

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="roberta.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

INFO:tensorflow:Assets written to: /tmp/tmphxx9i1ig/assets


INFO:tensorflow:Assets written to: /tmp/tmphxx9i1ig/assets
INFO:absl:Using experimental converter: If you encountered a problem please file a bug. You can opt-out by setting experimental_new_converter=False


In [ ]:
%%time
interpreter.set_tensor(input_details[0]['index'], inputs['input_ids'])
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

Reference:
* https://discuss.huggingface.co/t/how-can-we-test-transformer-models-after-converting-it-to-tflite-format/1670/4
* https://www.tensorflow.org/lite/guide/inference
* https://github.com/huggingface/tflite-android-transformers/blob/master/models_generation/distilbert.py
* https://www.tensorflow.org/lite/performance/post_training_quantization
* https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_lite/tflite_c02_transfer_learning.ipynb
* https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_lite/tflite_c04_exercise_convert_model_to_tflite_solution.ipynb